In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dataset_path = "../input/global-wheat-detection/"
trainImg_path = dataset_path+"train/"

# def class_to_id(class_name):
#     return{
#         'usask_1':0,
#         'arvalis_1': 1,
#         'inrae_1': 2,
#         'ethz_1': 3,
#         'arvalis_3': 4,
#         'rres_1': 5,
#         'arvalis_2': 6,
#     }[class_name]


# Training

In [ ]:
import os

if not os.path.exists('training_data'):
    os.makedirs('training_data')

In [ ]:
# import csv
# import shutil as sh

# # 開啟 CSV 檔案
# n = 0
# with open(dataset_path+'train.csv', newline='') as csvfile:

#   # 讀取 CSV 檔案內容
#   rows = csv.reader(csvfile)

#   # 以迴圈輸出每一列
#   for row in rows:
#     imgId = row[0]
#     if imgId=='image_id':
#         continue
#     print(imgId)
#     width = row[1]
#     height = row[2]
    
# #     print(row[3])
#     bbox = []
#     b = ''
#     for r in row[3]:
#         r = ord(r)
#         if (r>=48 and r<=57) or r==46:
#             b += chr(r)
#         elif r ==44 or r==93:
#             bbox.append(float(b))
#             b = ''
#     class_name = row[4]
    
# #     print(imgId, width, height, bbox, class_name, classId)
    
#     _l = int(bbox[0])
#     _t = int(bbox[1])
    
#     x_center = (_l+int(bbox[2])/2)/int(width)
#     y_center = (_t+int(bbox[3])/2)/int(height)
#     bbox_width = int(bbox[2])/int(width)
#     bbox_height = int(bbox[3])/int(height)
    
# #     classId = class_to_id(class_name)
    
#     s = '0 '+str(x_center)+' '+str(y_center)+' '+str(bbox_width)+' '+str(bbox_height)+'\n'
# #     print(s)

#     fp = open('training_data/'+str(imgId)+'.txt', 'a')
#     fp.write(s)
#     fp.close()
#     if not os.path.isfile('training_data/{}.jpg'.format(imgId)):
#         sh.copy("../input/global-wheat-detection/train/{}.jpg".format(imgId),'training_data/{}.jpg'.format(imgId))

In [ ]:
# !ls ./training_data

In [ ]:
f = open('wheat.yaml', 'w')

f.write('train: ./training_data \n')
f.write('val: ./training_data \n')
f.write('test: ./testing_data \n')
f.write('\n')
f.write('nc: 1 \n')
f.write('\n')
f.write("names: ['wheat'] \n")
f.close()

In [ ]:
cat wheat.yaml

In [ ]:
# cp ../input/yolov5train/models/yolov5m.yaml yolov5m_wheat.yaml

In [ ]:
# f = open('../input/yolov5train/models/yolov5m.yaml', 'r')

# s = f.readlines()
# s[1] = "nc: 7  # number of classes\n"
# print(s[1])

# fw = open('yolov5m_wheat.yaml', 'w')
# fw.writelines(s)
# fw.close()

In [ ]:
# !cat yolov5m_wheat.yaml

In [ ]:
# ls ../input/global-wheat-detection/test

In [ ]:
# !python ../input/yolov5code/yolov5-master/train.py --device 0 --img-size 320 --batch-size 16 --epochs 1 --data wheat.yaml --cfg yolov5m_wheat.yaml

In [ ]:
!cp -r ../input/global-wheat-detection/test testing_data

In [ ]:
# !python ../input/yolov5code/yolov5-master/test.py --task test --device 0 --weights ../input/yolov5m-320-16-50/best.pt --data wheat.yaml --img-size 320 --batch-size 16 --save-json --save-txt --name result

In [ ]:
# !cat runs/test/result/best_predictions.json

# Predict

In [ ]:
# def detect():
#     source = '../input/global-wheat-detection/test/'
#     weights = '../input/yolov5m-320-160-50-wheat1/best.pt'
# #     if not os.path.exists(weights):
# #         weights = '../input/bestweight/best_yolov5x_fold0.pt'
#     imgsz = 1024
#     conf_thres = 0.5
#     iou_thres = 0.6
#     is_TTA = True
    
#     imagenames =  os.listdir(source)
    
#     device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

#     # Load model
#     model = torch.load(weights, map_location=device)['model'].float()  # load to FP32
#     model.to(device).eval()
    
#     dataset = LoadImages(source, img_size=imgsz)

#     results = []
#     fig, ax = plt.subplots(5, 2, figsize=(30, 70))
#     count = 0
#     # img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
#     #for path, img, im0s, _ in dataset:
#     for name in imagenames:
#         image_id = name.split('.')[0]
#         im01 = cv2.imread('%s/%s.jpg'%(source,image_id))  # BGR
#         assert im01 is not None, 'Image Not Found '
#         # Padded resize
#         im_w, im_h = im01.shape[:2]
#         if is_TTA:
#             enboxes = []
#             enscores = []
#             for i in range(4):
#                 im0 = TTAImage(im01, i)
#                 boxes, scores = detect1Image(im0, imgsz, model, device, conf_thres, iou_thres)
#                 for _ in range(3-i):
#                     boxes = rotBoxes90(boxes, im_w, im_h)
                    
#                 if 1: #i<3:
#                     enboxes.append(boxes)
#                     enscores.append(scores) 
#             boxes, scores = detect1Image(im01, imgsz, model, device, conf_thres, iou_thres)
#             enboxes.append(boxes)
#             enscores.append(scores)

#             boxes, scores, labels = run_wbf(enboxes, enscores, image_size = im_w, iou_thr=0.6, skip_box_thr=0.5)
#             boxes = boxes.astype(np.int32).clip(min=0, max=im_w)
#         else:
#             boxes, scores = detect1Image(im01, imgsz, model, device, conf_thres, iou_thres)

#         boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
#         boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
        
#         boxes = boxes[scores >= 0.05].astype(np.int32)
#         scores = scores[scores >=float(0.05)]
#         if count<10:
#             #sample = image.permute(1,2,0).cpu().numpy()
#             for box, score in zip(boxes,scores):
#                 cv2.rectangle(im0,
#                               (box[0], box[1]),
#                               (box[2]+box[0], box[3]+box[1]),
#                               (220, 0, 0), 2)
#                 cv2.putText(im0, '%.2f'%(score), (box[0], box[1]), cv2.FONT_HERSHEY_SIMPLEX ,  
#                    0.5, (255,255,255), 2, cv2.LINE_AA)
#             ax[count%5][count//5].imshow(im0)
#             count+=1
            
#         result = {
#             'image_id': image_id,
#             'PredictionString': format_prediction_string(boxes, scores)
#         }

#         results.append(result)
#     return results

# Detect

In [ ]:
!rm submission.csv

# Change This!!!!

In [ ]:
!python ../input/yolov5code/yolov5-master/detect.py --source ../input/global-wheat-detection/test --device 0 --weight ../input/yolov5l-160-32-50-wheat/color.pt --conf 0.3 --save-txt --save-conf

In [ ]:
!ls

In [ ]:
# import os
# import cv2
# import csv

# def submit(l_path, test_path):
#     label_path = l_path
#     label_files = list(sorted(os.listdir(label_path)))
# #     print(label_files)
    
#     csvfile = open('submission.csv', 'a')
#     writer = csv.writer(csvfile)
#     writer.writerow(['image_id', 'PredictionString'])

#     for file in label_files:
#         f = open(label_path+file,'r')
#         contents = f.readlines()
        
#         img_name = file.replace('.txt', '.jpg')
#         print(img_name)
#         im = cv2.imread(test_path+img_name)
#         h, w, c = im.shape
#         print(h, w, c)
        
#         PredictionString = ''
#         for content in contents:
#             content = content.replace('\n','')
#             c = content.split(' ')
#             print(c)
            
#             w_center = w*float(c[1])
#             h_center = h*float(c[2])
#             width = w*float(c[3])
#             height = h*float(c[4])
#             left = int(w_center - width/2)
# #             right = int(w_center + width/2)
#             top = int(h_center - height/2)
# #             bottom = int(h_center + height/2)
#             score = float(c[5])
    
#             s = str(score)+' '+str(left)+' '+str(top)+' '+str(round(width))+' '+str(round(height))+' '
#             PredictionString += s
# #             print(score, left, top, round(width), round(height))
#         print(PredictionString)
#         imgId = file.replace('.txt', '')
#         writer.writerow([imgId, PredictionString])
        


# label_path = './runs/detect/exp/labels/'
# testImg_path = '../input/global-wheat-detection/test/'
# submit(label_path, testImg_path)

In [ ]:
!cat ./submission.csv

In [ ]:
import pandas as pd # 引用套件並縮寫為 pd  
df = pd.read_csv('submission.csv')  
print(df)  

# RCNN

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        
        os.path.join(dirname, filename)

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import cv2
from torchvision import datasets,transforms
from glob import glob
import os
from PIL import Image
from matplotlib import patches
from torch.utils.data import Dataset
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data import DataLoader
#Albumentation
import albumentations as al
from albumentations.pytorch.transforms import ToTensorV2

train_dir ='/kaggle/input/global-wheat-detection/train/'
test_dir = '/kaggle/input/global-wheat-detection/test/'
train = pd.read_csv('../input/global-wheat-detection/train.csv') 
weights = '../input/fasterrcnn-resnet50-fpn/fasterrcnn_resnet50_fpn (1).pth'

In [ ]:
sample = pd.read_csv('../input/global-wheat-detection/sample_submission.csv')
sample.shape

In [ ]:
sample

In [ ]:
class Wheatdatasets(Dataset):
    
    def __init__(self,dataframe,image_dir,transforms = None):
        
        super().__init__()
    
        self.image_id = dataframe['image_id'].unique()
        self.df = dataframe
        self.img_dir = image_dir
        self.transforms = transforms
        
    def __getitem__(self,index:int):
        
        image_id = self.image_id[index]
        record = self.df[self.df['image_id']==image_id]
        
        image = cv2.imread(self.img_dir+image_id+'.jpg',cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB).astype(np.float32)
        image = image/255.0
       
        if self.transforms:
            sample ={
                'image':image
            }
            sample =self.transforms(**sample)
            image = sample['image']
            
            return image,image_id
        
    def __len__(self) -> int:
        return self.image_id.shape[0]

In [ ]:
def get_transform():
    return al.Compose([
        ToTensorV2(p=1.0)
    ])

In [ ]:
#Create the Model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = False,pretrained_backbone=False)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
num_class = 2 #wheats and background

in_feature = model.roi_heads.box_predictor.cls_score.in_features

model.roi_heads.box_predictor = FastRCNNPredictor(in_feature,num_class) #changin the pretrained head with a new one

model.load_state_dict(torch.load(weights))
model.eval()

In [ ]:
!ls ../input

In [ ]:
x = model.to(device)

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))


test_dataset = Wheatdatasets(sample,test_dir,get_transform())



test_dataloader = DataLoader(
test_dataset,
batch_size=4,
shuffle =False,
num_workers =4,drop_last = False,
collate_fn = collate_fn)




In [ ]:
def format_string(boxes,score):
    pred_strings = []
    for j in zip(score,boxes):
        
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))
        
    return " ".join(pred_strings)
        

In [ ]:
detection_threshold = 0.5
results = []

for images, image_ids in test_dataloader:

    images = list(image.to(device) for image in images)
    outputs = model(images)

    for i, image in enumerate(images):

        boxes = outputs[i]['boxes'].data.cpu().numpy()
        scores = outputs[i]['scores'].data.cpu().numpy()
        
        boxes = boxes[scores >= detection_threshold].astype(np.int32)
        scores = scores[scores >= detection_threshold]
        image_id = image_ids[i]
        
        boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
        
        result = {
            'image_id': image_id,
            'PredictionString': format_string(boxes, scores)
        }

        
        results.append(result)

In [ ]:
results[:2]

In [ ]:
test_df = pd.DataFrame(results, columns=['image_id', 'PredictionString'])
test_df.head()

In [ ]:
# sample = images[0].permute(1,2,0).cpu().numpy()
# boxes = outputs[0]['boxes'].data.cpu().numpy()
# score = outputs[0]['scores'].data.cpu().numpy()

# boxes = boxes[score >= detection_threshold].astype(np.int32)

In [ ]:
# fig, ax = plt.subplots(1, 1, figsize=(16, 8))

# for box in boxes:
#     cv2.rectangle(sample,
#                   (box[0], box[1]),
#                   (box[2], box[3]),
#                   (220, 0, 0), 2)
    
# ax.set_axis_off()
# ax.imshow(sample)

In [ ]:
test_df.to_csv('submission.csv', index=False)

In [ ]:
test_df

In [ ]:
img_id = list(test_df['image_id'])
print(img_id)

In [ ]:
# df['image_id']
# # df = df.drop([0, 1])
# # df.reset_index(drop='true')

In [ ]:
# test_df.loc[test_df['image_id'] == i]

In [ ]:
for i in img_id:
    q = df.loc[df['image_id'] == i]
    if q.empty:
        print(i)
        p = test_df.loc[test_df['image_id'] == i]['PredictionString'].item()
        df2 = pd.DataFrame([[i, p]], columns=['image_id','PredictionString'])
        print(df2)
        df = df.append(df2)

In [ ]:
df.to_csv('submission.csv', index=False)

In [ ]:
# !cat 'submission.csv'